In [85]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',None)

In [86]:
movies_df = pd.read_csv('../input/movies_metadata.csv')
ratings_df = pd.read_csv('../input/ratings_small.csv')
casts_df = pd.read_csv('../input/cast.csv').sample(frac=0.2, random_state=42)

In [87]:
def unpac_column_list(df, column_name):
    df[column_name] = df[column_name].apply(lambda x: eval(x))
    return df

In [88]:
def rename_column(df, old_column, new_column):
    data = df.rename(columns={old_column:new_column})
    return data

In [89]:
movies_df = unpac_column_list(movies_df, 'genres')

In [90]:
genres_df = movies_df[['genres', 'id']]
genres_df = rename_column(genres_df, 'id', 'movie_id')

In [91]:
def unpac_genre_dict(df, column_name):
    unpac_dict = df.explode(column_name)
    column_df = pd.concat([unpac_dict.drop([column_name], axis=1), unpac_dict[column_name].apply(pd.Series)], axis=1)
    return column_df

In [92]:
unpacked_genre_df = unpac_genre_dict(genres_df, 'genres')

In [93]:
unpacked_genre_df = rename_column(unpacked_genre_df, 'id','genre_id')

In [94]:
def drop_column(df, column_list):
    data = df.drop(columns =column_list)
    return data

In [95]:
column_to_drop = [0]

In [96]:
unpacked_genre_df = drop_column(unpacked_genre_df, column_to_drop)

In [97]:
def merge_two_dfs(df1, df2, left, right, how):
    data = pd.merge(df1, df2, left_on=left, right_on=right, how=how)
    return data

In [98]:
def to_numeric(data, column):
    data[column] = pd.to_numeric(data[column], errors='coerce', downcast='integer')
    data = data.fillna(0)
    data[column] = data[column].astype(int)
    return data

In [99]:
movies_genre_df = merge_two_dfs(movies_df, unpacked_genre_df, 'id', 'movie_id', 'left')

In [100]:
movies_genre_df = to_numeric(movies_genre_df, column='movie_id')

In [101]:
movies_genre_df_copy = movies_genre_df.copy()

In [102]:
columns_to_drop = ['genres', 'homepage', 'imdb_id', 'poster_path', 'spoken_languages', 
                   'tagline', 'id', 'production_companies', 'production_countries', 'belongs_to_collection']

In [103]:
movies_genre_df_copy = drop_column(movies_genre_df_copy, columns_to_drop)

In [104]:
movies_genre_rating_df = merge_two_dfs(movies_genre_df_copy, ratings_df, 'movie_id', 'movieId', how= 'inner')

In [105]:
columns_to_drop = ['overview', 'original_title', 'genre_id', 'movieId', 'timestamp']

In [106]:
movies_genre_rating_df = drop_column(movies_genre_rating_df, columns_to_drop)

In [107]:
movies_genre_rating_casts_df = merge_two_dfs(movies_genre_rating_df, casts_df, 'movie_id', 'movie_id', how= 'inner')

In [108]:
movies_genre_rating_casts_df = rename_column(movies_genre_rating_casts_df, 'name_x', "genre_name")
movies_genre_rating_casts_df = rename_column(movies_genre_rating_casts_df, 'name_y', "cast_name")

In [109]:
columns_to_drop = ['credit_id', 'id', 'order', 'profile_path', 'cast_id', 'adult']

In [110]:
movies_genre_rating_casts_df = drop_column(movies_genre_rating_casts_df, columns_to_drop)

In [111]:
order_column_list = ['budget', 'original_language', 'popularity', 'release_date',
       'revenue', 'runtime', 'status', 'movie_id', 'title', 'video', 'vote_average',
       'vote_count', 'genre_name', 'userId', 'cast_name',
       'character', 'gender', 'rating']

In [112]:
movies_genre_rating_casts_df = movies_genre_rating_casts_df.reindex(columns=order_column_list)

In [113]:
def get_missing_values(df):
        features_with_missing_values = [features for features in df.columns if
                                        df[features].isnull().sum()>0]
        for features in features_with_missing_values:
            print(features, np.round(df[features].isnull().sum(), 4), "missing Values")

In [114]:
missing_values = get_missing_values(movies_genre_rating_casts_df)

cast_name 71 missing Values
character 3670 missing Values
gender 71 missing Values


In [115]:
def fill_missing_cat(data,col,value):
    data[col] = data[col].fillna(value)
    return data

In [116]:
list_of_columns = ['cast_name', 'character']

In [117]:
movies_genre_rating_casts_df = fill_missing_cat(movies_genre_rating_casts_df, list_of_columns, 'Unknown')
movies_genre_rating_casts_df = fill_missing_cat(movies_genre_rating_casts_df, 'gender', 0)

In [118]:
missing_values = get_missing_values(movies_genre_rating_casts_df)

In [119]:
string_column_list = ['original_language', 'status', 'title', 'video', 'genre_name', 'cast_name', 'character']
float_column_list = ['budget', 'popularity']
datetime_column_list = ['release_date']


In [120]:
def convert_object(data, column_list, data_type):
    data[column_list] = data[column_list].astype(data_type)
    return data
    

In [121]:
movies_genre_rating_casts_df = convert_object(movies_genre_rating_casts_df, float_column_list, float)

In [122]:
movies_genre_rating_casts_df = convert_object(movies_genre_rating_casts_df, string_column_list, str)

In [123]:
movies_genre_rating_casts_df['status'] = movies_genre_rating_casts_df['status'].replace('0', "Uknown")

In [124]:
movies_genre_rating_casts_df['genre_name'] = movies_genre_rating_casts_df['genre_name'].replace('0', "Uknown")

In [125]:
movies_genre_rating_casts_df = movies_genre_rating_casts_df[movies_genre_rating_casts_df['release_date']!=0]

In [126]:
movies_genre_rating_casts_df[['release_year', 'release_month', 'release_day']] = movies_genre_rating_casts_df['release_date'].str.split('-', 2).tolist()

In [127]:
movies_genre_rating_casts_df = drop_column(movies_genre_rating_casts_df, 'release_date')

In [128]:
movies_genre_rating_casts_df = movies_genre_rating_casts_df.set_index('release_year')

In [129]:
movies_genre_rating_casts_df = movies_genre_rating_casts_df.drop_duplicates()

In [130]:
movies_genre_rating_casts_df.to_csv('../output/cleaned_data.csv')

In [139]:
movies_genre_rating_casts_df.head(10)

,budget,original_language,popularity,revenue,runtime,status,movie_id,title,video,vote_average,vote_count,genre_name,userId,cast_name,character,gender,rating,release_month,release_day
release_year,,,,,,,,,,,,,,,,,,,
1995,60000000.0,en,17.924927,187436818.0,170.0,Released,949,Heat,False,7.7,1886.0,Action,23,Tone Loc,Richard Torena,2.0,3.5,12,15
1995,60000000.0,en,17.924927,187436818.0,170.0,Released,949,Heat,False,7.7,1886.0,Action,23,Kai Soremekun,Prostitute,1.0,3.5,12,15
1995,60000000.0,en,17.924927,187436818.0,170.0,Released,949,Heat,False,7.7,1886.0,Action,23,Susan Traylor,Elaine Cheritto,1.0,3.5,12,15
1995,60000000.0,en,17.924927,187436818.0,170.0,Released,949,Heat,False,7.7,1886.0,Action,23,Kim Staunton,Lillian,1.0,3.5,12,15
1995,60000000.0,en,17.924927,187436818.0,170.0,Released,949,Heat,False,7.7,1886.0,Action,23,Paul Herman,Sergeant Heinz,2.0,3.5,12,15
1995,60000000.0,en,17.924927,187436818.0,170.0,Released,949,Heat,False,7.7,1886.0,Action,23,Hank Azaria,Alan Marciano,2.0,3.5,12,15
1995,60000000.0,en,17.924927,187436818.0,170.0,Released,949,Heat,False,7.7,1886.0,Action,23,Farrah Forke,Claudia,1.0,3.5,12,15
1995,60000000.0,en,17.924927,187436818.0,170.0,Released,949,Heat,False,7.7,1886.0,Action,23,Kimberly Flynn,Casals' Date,1.0,3.5,12,15
1995,60000000.0,en,17.924927,187436818.0,170.0,Released,949,Heat,False,7.7,1886.0,Action,23,Philip Ettington,Ellis (uncredited),2.0,3.5,12,15
